In [1]:
from os.path import join
from os import makedirs
import csv
import SimpleITK as sitk
from tqdm import tqdm
import numpy as np
import scipy.ndimage.morphology
from skimage import measure, filters
from metrics import dc, jc, assd, sensitivity, specificity, precision, recall, intersection_percentage
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from nodule_segmentation import slice_segment
import torch

In [2]:
def threshold_mask(raw_output, threshold):
    if threshold == 0:
        try:
            threshold = filters.threshold_otsu(raw_output)
        except:
            threshold = 0.5

    # print('\tThreshold: {}'.format(threshold))

    raw_output[raw_output > threshold] = 1
    raw_output[raw_output < 1] = 0

    all_labels = measure.label(raw_output)
    props = measure.regionprops(all_labels)
    props.sort(key=lambda x: x.area, reverse=True)
    thresholded_mask = np.zeros(raw_output.shape)

    if len(props) >= 2:
        if props[0].area / props[1].area > 5:  # if the largest is way larger than the second largest
            thresholded_mask[all_labels == props[0].label] = 1  # only turn on the largest component
        else:
            thresholded_mask[all_labels == props[0].label] = 1  # turn on two largest components
            thresholded_mask[all_labels == props[1].label] = 1
    elif len(props):
        thresholded_mask[all_labels == props[0].label] = 1

    thresholded_mask = scipy.ndimage.morphology.binary_fill_holes(thresholded_mask).astype(np.uint8)

    return thresholded_mask

In [7]:
def np2Png(np_arr, target_path):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr) if np.max(np_arr) > 0 else 1
    scaled_np_arr = (np_arr - min_value) / (max_value - min_value) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slices = Image.fromarray(scaled_np_arr, mode='L')
    slices.save(target_path)

In [2]:
outdir = "aug_new"
# from test_result import test_result
# test_result(True, outdir, True)

In [5]:
# from dsetsFullCT import Luna2dSegmentationDataset
# from torch.utils.data import ConcatDataset, DataLoader
# from tqdm import tqdm
# all_ds = Luna2dSegmentationDataset(
#     val_stride=10,
#     set_class="All",
#     contextSlices_count=3,
# )
# # print(len(all_ds))
# csv_filename = "./result/all_nodule.csv"
# with open(csv_filename, 'w', newline='') as csvfile:
#     header = ["UID", "NDX"]
#     csv_writer = csv.writer(csvfile)
#     csv_writer.writerow(header)
#     batch_size = 1

#     all_dl = DataLoader(
#         all_ds,
#         batch_size=batch_size,
#         num_workers=16,
#         pin_memory=True,
#     )
#     with torch.no_grad():
#         pred_mat = []
#         for data in tqdm(all_dl, desc="Nodule"):
#             input_t, label_t, series_list, _slice_ndx_list = data
#             row = [series_list[0], _slice_ndx_list[0].item()]
#             csv_writer.writerow(row)

# #     batch_size = 1
# #     batch_size *= torch.cuda.device_count()
# #     csv_filename = "D://LUNA//udet//result//result.csv"

# #     test_dl = DataLoader(
# #         test_ds,
# #         batch_size=batch_size,
# #         num_workers=16,
# #         pin_memory=True,
# #     )
# #     segmentation_model.to(device)
# #     testMetrics_t = doTesting(epoch_ndx, test_dl, segmentation_model, device, csv_filename)
# #     logMetrics(epoch_ndx, 'test', testMetrics_t)

In [3]:
# pip install pandas 
import pandas as pd
from dsetsFullCT import Luna2dSegmentationDataset

# Read the csv file
result_df = pd.read_csv('C://LUNA//udet//result//{}//result.csv'.format(outdir))
ann_df = pd.read_csv('C://Users//oplab//Desktop//Luna16_data//ann.csv')
# result_df.head()
# ann_df.head()

# Merge the dataframes on the 'UID' column
merged_df = pd.merge(result_df, ann_df, on='UID')

filtered_df = merged_df.loc[(merged_df['NDX'] >= merged_df['Start']) & (merged_df['NDX'] <= merged_df['End'])]

# filtered_df

filtered_df.to_csv('.//result//{}//match_nodule.csv'.format(outdir), index=False)
# # print(merged_df.head())
filtered_df

# # Function to identify the corresponding 'NID' based on 'Start' and 'End'
# def get_nid(row):
#     ndx = row['NDX']
#     return ann_df[(ann_df['UID'] == row['UID']) & (ann_df['Start'] <= ndx) & (ann_df['End'] >= ndx)]['NID'].values

# # Apply the function to create the 'NID' column
# merged_df['NID'] = merged_df.apply(get_nid, axis=1)

# # Flatten the 'NID' column (since it may contain arrays)
# merged_df['NID'] = merged_df['NID'].apply(lambda x: x[0] if len(x) > 0 else None)

# merged_df.head()

# # Save the updated dataframe to a new CSV file
# merged_df.to_csv('result_with_nid.csv', index=False)

,UID,NDX,Dice Loss,Dice,Jaccard,Precision,Recall,Sensitivity,Specificity,NID,Start,End,StartRow,EndRow,StartCol,EndCol,Diameter
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,88,0.259341,0.708333,0.548387,0.686869,0.731183,0.731183,0.999882,1,88,90,264,276,377,393,10.915156
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,89,0.143298,0.858824,0.752577,0.901235,0.820225,0.820225,0.999969,1,88,90,264,276,377,393,10.915156
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,90,0.185000,0.756757,0.608696,0.717949,0.800000,0.800000,0.999958,1,88,90,264,276,377,393,10.915156
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.107109359065...,49,0.973929,0.000000,0.000000,0.000000,0.000000,0.000000,0.999935,2,49,69,138,175,104,147,29.524875
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.107109359065...,50,0.996554,0.000000,0.000000,0.000000,0.000000,0.000000,0.999821,2,49,69,138,175,104,147,29.524875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...,215,0.217323,0.740072,0.587393,0.740072,0.740072,0.740072,0.999725,19,205,219,361,383,388,412,15.799870
2462,1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...,216,0.175305,0.809639,0.680162,0.971098,0.694215,0.694215,0.999981,19,205,219,361,383,388,412,15.799870
2467,1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...,217,0.940425,0.116402,0.061798,1.000000,0.061798,0.061798,1.000000,19,205,219,361,383,388,412,15.799870
2472,1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...,218,0.997382,0.000000,0.000000,0.000000,0.000000,0.000000,0.999687,19,205,219,361,383,388,412,15.799870


In [4]:
def create_bbox_mask(start_row, start_col, end_row, end_col, height=512, width=512):
    mask = torch.zeros((height, width), dtype=torch.uint8)
    
    # Ensure the bounding box coordinates are within the image boundaries
    start_row = max(0, start_row)
    start_col = max(0, start_col)
    end_row = min(height, end_row)
    end_col = min(width, end_col)
    
    # Set the region inside the bounding box to 1
    mask[start_row:end_row, start_col:end_col] = 1
    
    return mask

In [7]:
grouped = filtered_df.groupby(['UID', 'NID'])
i = 0
threshold = 0.5
iou_3d = []
dc_3d = []
ip_3d = []
gt_arr = []
pre_arr = []
for (uid, nid), group_df in grouped:
    # print(f"Group with UID={uid} and NID={nid}:")
    # print(group_df)
    # print('\n')
    gt_list = []
    pre_list = []
    for index, row in group_df.iterrows():
        # print(row["UID"], row["NDX"])
        # print(row)
        start_row, end_row, start_col, end_col = row["StartRow"], row["EndRow"], row["StartCol"], row["EndCol"]
        bbox_mask = create_bbox_mask(start_row, start_col, end_row, end_col)
        # Load the saved tensors
        loaded_data = torch.load('C://LUNA//udet//result//{}//output//{}_{}.pth'.format(outdir, row["UID"], row["NDX"]))

        # Retrieve label_g and prediction_g from the loaded data
        label_g = loaded_data['label_g'].squeeze(1)
        prediction_g = loaded_data['prediction_g'].squeeze(1)
        prediction_g = (torch.from_numpy(threshold_mask(prediction_g.cpu(), threshold)).to("cuda") & bbox_mask.to("cuda"))
        # print(label_g.size())
        # print(prediction_g.size())
        gt_list.append(label_g)
        pre_list.append(prediction_g)
        gt_arr.append(label_g)
        pre_arr.append(prediction_g)
    gt_t = torch.stack(gt_list, dim=1).squeeze()
    pre_t = torch.stack(pre_list, dim=1).squeeze()
    dc_r = dc(pre_t, gt_t)
    jc_r = jc(pre_t, gt_t)
    ip_r = intersection_percentage(pre_t, gt_t)
    dc_3d.append(dc_r)
    iou_3d.append(jc_r)
    ip_3d.append(ip_r)
    # print(result)
    # if i > 1:
    # break
    i += 1

C:\Users\oplab\AppData\Local\Temp\ipykernel_4876\2846807596.py:27: DeprecationWarning: Please use `binary_fill_holes` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  thresholded_mask = scipy.ndimage.morphology.binary_fill_holes(thresholded_mask).astype(np.uint8)


In [23]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Assuming y_true contains true labels for the entire testing dataset and y_pred contains predicted probabilities
y_true = torch.stack(gt_arr).squeeze().cpu().numpy().astype(bool)
y_pred = torch.stack(pre_arr).squeeze().cpu().numpy().astype(bool)
# print(y_true)
# print(y_pred.astype(bool))
fpr, tpr, thresholds = roc_curve(y_true.ravel(), y_pred.ravel())
roc_auc = auc(fpr, tpr)

In [29]:
# print(len(gt_arr))
# print(gt_arr[0].size())
# print(type(y_true))

<class 'numpy.ndarray'>


In [10]:
# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve for the Entire Testing Dataset')
plt.legend(loc='lower right')
plt.savefig(".//result//{}//ROC.png".format(outdir), format='png', bbox_inches='tight')

In [10]:
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

# Assuming y_true contains true labels for the entire testing dataset and y_pred contains predicted probabilities
# Ensure y_true is in a suitable format (binary or multiclass)
# If it's multiclass, you may need to convert it to a one-hot encoded format

# Example conversion for multiclass labels to one-hot encoded format
# y_true_one_hot = np.eye(num_classes)[y_true]

precision, recall, thresholds = precision_recall_curve(y_true.ravel(), y_pred.ravel())
pr_auc = auc(recall, precision)

# Plot the PR curve
plt.figure()
plt.plot(recall, precision, color='darkorange', lw=2, label=f'AUC = {pr_auc:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for the Entire Testing Dataset')
plt.legend(loc='lower right')
plt.savefig(".//result//{}//PR-curve.png".format(outdir), format='png', bbox_inches='tight')

In [45]:
def calculate_f1_score(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1_score

In [44]:
y_true_t = torch.from_numpy(y_true.astype(int))
y_pred_t = torch.from_numpy(y_pred.astype(int))
tp_sum = 0
fn_sum = 0
fp_sum = 0
i = 0
for (uid, nid), group_df in grouped:
    if iou_3d[i] < 0.5:
        continue
    for index, row in group_df.iterrows():
        predictionBool_g = y_true_t[i]
        label_g = y_pred_t[i]
        # print(predictionBool_g.size())
        tp = (     predictionBool_g *  label_g).sum(dim=[0,1])
        fn = ((1 - predictionBool_g) *  label_g).sum(dim=[0,1])
        fp = (     predictionBool_g * (1 - label_g)).sum(dim=[0,1])
        tp_sum += tp
        fn_sum += fn
        fp_sum += fp
    i += 1
print(tp_sum)
print(fn_sum)
print(fp_sum)

tensor(1793)
tensor(196)
tensor(425)


In [50]:
tp = tp_sum.item()
fn = fn_sum.item()
fp = fp_sum.item()
# print(tp)
calculate_f1_score(tp, fp, fn)

0.852388875683385

In [39]:
print(len(iou_3d))
print(len(dc_3d))
print(len(ip_3d))

127
127
127


In [13]:
filtered_df.loc[:, "SLICES"] = filtered_df["End"] - filtered_df["Start"]

C:\Users\oplab\AppData\Local\Temp\ipykernel_23492\199489479.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, "SLICES"] = filtered_df["End"] - filtered_df["Start"]


In [14]:
filtered_df

,UID,NDX,Dice Loss,Dice,Jaccard,Precision,Recall,Sensitivity,Specificity,NID,Start,End,StartRow,EndRow,StartCol,EndCol,Diameter,SLICES
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,88,0.251397,0.772152,0.628866,0.938462,0.655914,0.655914,0.999985,1,88,90,264,276,377,393,10.915156,2
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,89,0.203602,0.787500,0.649485,0.887324,0.707865,0.707865,0.999969,1,88,90,264,276,377,393,10.915156,2
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,90,0.289474,0.684932,0.520833,0.657895,0.714286,0.714286,0.999950,1,88,90,264,276,377,393,10.915156,2
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.107109359065...,49,0.965116,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2,49,69,138,175,104,147,29.524875,20
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.107109359065...,50,0.995746,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2,49,69,138,175,104,147,29.524875,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...,215,0.267927,0.629412,0.459227,0.531017,0.772563,0.772563,0.999278,19,205,219,361,383,388,412,15.799870,14
2462,1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...,216,0.154041,0.871111,0.771654,0.942308,0.809917,0.809917,0.999954,19,205,219,361,383,388,412,15.799870,14
2467,1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...,217,0.253021,0.855305,0.747191,1.000000,0.747191,0.747191,1.000000,19,205,219,361,383,388,412,15.799870,14
2472,1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...,218,0.997481,0.000000,0.000000,0.000000,0.000000,0.000000,0.999653,19,205,219,361,383,388,412,15.799870,14


In [15]:
average_values = filtered_df.groupby(['UID', 'NID'])[['Dice', 'Jaccard', 'Diameter', 'SLICES']].mean().reset_index()
average_values["IOU"] = iou_3d
average_values["DICE"] = dc_3d
average_values["IP"] = ip_3d
print(average_values)

                                                   UID  NID      Dice  \
0    1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...    1  0.748194   
1    1.3.6.1.4.1.14519.5.2.1.6279.6001.107109359065...    2  0.493662   
2    1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295...    1  0.813725   
3    1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295...    4  0.251341   
4    1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295...    5  0.710181   
..                                                 ...  ...       ...   
122  1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...    1  0.814824   
123  1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...    9  0.820620   
124  1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...   10  0.738293   
125  1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...   13  0.836956   
126  1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...   19  0.639012   

      Jaccard   Diameter  SLICES       IOU      DICE        IP  
0    0.599728  10.915156     2.0  0.615702  0.762148  0.68

In [16]:
average_values.to_csv('./result/{}/average_nodule.csv'.format(outdir), index=False)

In [17]:
outdir = "origin"
average_values = pd.read_csv('./result/{}/average_nodule.csv'.format(outdir))

In [18]:
def get_plot(df, output_file='plot.png'):
    df['Row_ID'] = list(range(len(df)))
    plt.figure(figsize=(18, 6))  # Adjust the figure size as needed

    plt.subplot(1, 2, 1)
    plt.hist(df['Jaccard'], bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    plt.title('Avg IOU Distribution')
    plt.xlabel('Avg IOU')
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    plt.hist(df['Dice'], bins=20, color='salmon', edgecolor='black', alpha=0.7)
    plt.title('Dice Loss Distribution')
    plt.xlabel('Dice Loss')
    plt.ylabel('Frequency')

    plt.tight_layout()

    # Save the plot to a file
    plt.savefig(output_file)
    
def get_plot3d(df, output_file='plot.png'):
    df['Row_ID'] = list(range(len(df)))
    plt.figure(figsize=(18, 6))  # Adjust the figure size as needed
    
    plt.subplot(1, 2, 1)
    plt.hist(df['IOU'], bins=20, color='green', edgecolor='black', alpha=0.7)  # Adjust the color
    plt.title('3D IOU Distribution')
    plt.xlabel('3D IOU')
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    plt.hist(df['DICE'], bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    plt.title('3D Dice Distribution')
    plt.xlabel('3D Dice')
    plt.ylabel('Frequency')

    plt.tight_layout()

    # Save the plot to a file
    plt.savefig(output_file)
def get_plotip(df, output_file='plot.png'):
    df['Row_ID'] = list(range(len(df)))
    plt.figure(figsize=(18, 6))  # Adjust the figure size as needed
    
    plt.subplot(1, 1, 1)
    plt.hist(df['IP'], bins=20, color='green', edgecolor='black', alpha=0.7)  # Adjust the color
    plt.title('Intersection Percentage Distribution')
    plt.xlabel('Intersection Percentage')
    plt.ylabel('Frequency')

    plt.tight_layout()

    # Save the plot to a file
    plt.savefig(output_file)

# Example usage
get_plot(average_values, output_file='./result/{}/output_plot.png'.format(outdir))
get_plot3d(average_values, output_file='./result/{}/output_plot3d.png'.format(outdir))
get_plotip(average_values, output_file='./result/{}/output_plotip.png'.format(outdir))

In [18]:
jaccard_g6 = average_values[average_values['Jaccard'] > 0.6]
print(len(jaccard_g6))
jaccard_g5 = average_values[average_values['Jaccard'] > 0.5]
print(len(jaccard_g5))
jaccard_g4 = average_values[average_values['Jaccard'] > 0.4]
print(len(jaccard_g4))

72
104
119


In [19]:
def rid_of_outlier(df, column_name):
    Q1 = df[column_name].quantile(0.2)
    Q3 = df[column_name].quantile(0.8)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_no_outliers = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
    return df_no_outliers
def filter_result(threshold, csv_name, average_values, col_name, outdir):
    jaccard_gt = average_values[average_values[col_name] > threshold]
    jaccard_gt_wo = rid_of_outlier(jaccard_gt, "Diameter")
    print("count: ", len(jaccard_gt))
    print("max: ", jaccard_gt["Diameter"].max())
    print("min: ", jaccard_gt["Diameter"].min())
    print("mean: ", jaccard_gt["Diameter"].mean())
    print("max without outlier: ", jaccard_gt_wo["Diameter"].max())
    print("min without outlier: ", jaccard_gt_wo["Diameter"].min())
    print("mean without outlier: ", jaccard_gt_wo["Diameter"].mean())
    print("slices:")
    print("max: ", jaccard_gt["SLICES"].max())
    print("min: ", jaccard_gt["SLICES"].min())
    print("mean: ", jaccard_gt["SLICES"].mean())
    jaccard_gt.to_csv('./result/{}/{}_gt{}.csv'.format(outdir, col_name, csv_name), index=False)
    jaccard_gt_wo.to_csv('./result/{}/{}_gt{}_wo.csv'.format(outdir, col_name, csv_name), index=False)
    jaccard_lt = average_values[average_values[col_name] < threshold]
    jaccard_lt_wo = rid_of_outlier(jaccard_lt, "Diameter")
    print("count: ", len(jaccard_lt))
    print("diameter:")
    print("max: ", jaccard_lt["Diameter"].max())
    print("min: ", jaccard_lt["Diameter"].min())
    print("mean: ", jaccard_lt["Diameter"].mean())
    print("max without outlier: ", jaccard_lt_wo["Diameter"].max())
    print("min without outlier: ", jaccard_lt_wo["Diameter"].min())
    print("mean without outlier: ", jaccard_lt_wo["Diameter"].mean())
    print("slices:")
    print("max: ", jaccard_lt["SLICES"].max())
    print("min: ", jaccard_lt["SLICES"].min())
    print("mean: ", jaccard_lt["SLICES"].mean())
    jaccard_lt.to_csv('./result/{}/{}_lt{}.csv'.format(outdir, col_name, csv_name), index=False)
    jaccard_lt_wo.to_csv('./result/{}/{}_lt{}_wo.csv'.format(outdir, col_name, csv_name), index=False)

In [20]:
filter_result(0.5, "5", average_values, "DICE", outdir)

count:  110
max:  49.40658910877456
min:  5.524271728019902
mean:  13.882123064868173
max without outlier:  29.524874722621604
min without outlier:  5.524271728019902
mean without outlier:  11.028668776168601
slices:
max:  47.0
min:  1.0
mean:  6.3545454545454545
count:  17
diameter:
max:  42.66101291028559
min:  5.35484046506056
mean:  10.524113732463988
max without outlier:  13.85700550488813
min without outlier:  5.35484046506056
mean without outlier:  7.959170113494772
slices:
max:  19.0
min:  2.0
mean:  4.823529411764706


In [21]:
filter_result(0.5, "5", average_values, "Jaccard", outdir)

count:  93
max:  49.40658910877456
min:  5.668774979272418
mean:  13.339067503527659
max without outlier:  28.71092000000004
min without outlier:  5.668774979272418
mean without outlier:  10.955828715847348
slices:
max:  42.0
min:  1.0
mean:  6.064516129032258
count:  34
diameter:
max:  44.691324137073856
min:  5.35484046506056
mean:  13.688535081156312
max without outlier:  29.524874722621604
min without outlier:  5.35484046506056
mean without outlier:  9.89472295117565
slices:
max:  47.0
min:  1.0
mean:  6.382352941176471


In [22]:
filter_result(0.5, "5", average_values, "IOU", outdir)

count:  99
max:  49.40658910877456
min:  5.668774979272418
mean:  14.233306446956153
max without outlier:  32.53425553479171
min without outlier:  5.668774979272418
mean without outlier:  11.534452596865727
slices:
max:  47.0
min:  1.0
mean:  6.494949494949495
count:  28
diameter:
max:  42.66101291028559
min:  5.35484046506056
mean:  10.601647583525992
max without outlier:  16.861368839180606
min without outlier:  5.35484046506056
mean without outlier:  8.64077864253156
slices:
max:  20.0
min:  1.0
mean:  4.928571428571429


In [23]:
filter_result(0.5, "5", average_values, "IP", outdir)

count:  105
max:  49.40658910877456
min:  5.524271728019902
mean:  14.111930532224104
max without outlier:  32.53425553479171
min without outlier:  5.524271728019902
mean without outlier:  11.570376055758198
slices:
max:  47.0
min:  1.0
mean:  6.476190476190476
count:  22
diameter:
max:  42.66101291028559
min:  5.35484046506056
mean:  10.190489304720725
max without outlier:  14.04540436892935
min without outlier:  5.35484046506056
mean without outlier:  8.233419147719486
slices:
max:  19.0
min:  1.0
mean:  4.590909090909091
